### Challenge 30: Break an MD4 keyed MAC using length extension

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Second verse, same as the first, but use MD4 instead of SHA-1. Having done this attack once against SHA-1, the MD4 variant should take much less time; mostly just the time you'll spend Googling for an implementation of MD4.

<div class="alert alert-block alert-warning">

#### **You're thinking, why did we bother with this?**

Blame Stripe. In their second CTF game, the second-to-last challenge involved breaking an H(k, m) MAC with SHA1. Which meant that SHA1 code was floating all over the Internet. MD4 code, not so much.

</div>


</div>

In [1]:
from Crypto import Random
import cryptopals as cp
import md4

I used the pure Python3 MD4 implementation from [https://gist.github.com/BenWiederhake/eb6dfc2c31d3dc8c34508f4fd091cea9](https://gist.github.com/BenWiederhake/eb6dfc2c31d3dc8c34508f4fd091cea9)

In [2]:
# From  https://gist.github.com/BenWiederhake/eb6dfc2c31d3dc8c34508f4fd091cea9

import codecs
import struct

def leftrotate(i, n):
    return ((i << n) & 0xffffffff) | (i >> (32 - n))

def F(x, y, z):
    return (x & y) | (~x & z)

def G(x, y, z):
    return (x & y) | (x & z) | (y & z)

def H(x, y, z):
    return x ^ y ^ z

class MD4(object):
    def __init__(self, data=b''):
        self.remainder = data
        self.count = 0
        self.h = [
                0x67452301,
                0xefcdab89,
                0x98badcfe,
                0x10325476
                ]

    def _add_chunk(self, chunk):
        self.count += 1
        X = list( struct.unpack("<16I", chunk) + (None,) * (80-16) )
        h = [x for x in self.h]
        # Round 1
        s = (3, 7, 11, 19)
        for r in range(16):
            i = (16-r)%4
            k = r
            h[i] = leftrotate( (h[i] + F(h[(i+1)%4], h[(i+2)%4], h[(i+3)%4]) + X[k]) % 2**32, s[r%4] )
        # Round 2
        s = (3, 5, 9, 13)
        for r in range(16):
            i = (16-r)%4
            k = 4*(r%4) + r//4
            h[i] = leftrotate( (h[i] + G(h[(i+1)%4], h[(i+2)%4], h[(i+3)%4]) + X[k] + 0x5a827999) % 2**32, s[r%4] )
        # Round 3
        s = (3, 9, 11, 15)
        k = (0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15) #wish I could function
        for r in range(16):
            i = (16-r)%4
            h[i] = leftrotate( (h[i] + H(h[(i+1)%4], h[(i+2)%4], h[(i+3)%4]) + X[k[r]] + 0x6ed9eba1) % 2**32, s[r%4] )

        for i, v in enumerate(h):
            self.h[i] = (v + self.h[i]) % 2**32

    def add(self, data):
        message = self.remainder + data
        r = len(message) % 64
        if r != 0:
            self.remainder = message[-r:]
        else:
            self.remainder = b''
        for chunk in range(0, len(message)-r, 64):
            self._add_chunk( message[chunk:chunk+64] )
        return self

    def finish(self):
        l = len(self.remainder) + 64 * self.count
        self.add( b'\x80' + b'\x00' * ((55 - l) % 64) + struct.pack("<Q", l * 8) )
        out = struct.pack("<4I", *self.h)
        self.__init__()
        return out

<div class="alert alert-block alert-info">
    
To implement the attack, first write the function that computes the MD padding of an arbitrary message and verify that you're generating the same padding that your SHA-1 implementation is using. This should take you 5-10 minutes.
    
</div>

MD4 padding scheme is almost identical to SHA-1, so we can mostly re-use the ```compute_sha1_padding``` from Challenge #29.  It does look like it encodes message length in little endian instead of big endian.

<div class="alert alert-block alert-info">

Now, take the SHA-1 secret-prefix MAC of the message you want to forge --- this is just a SHA-1 hash --- and break it into 32 bit SHA-1 registers (SHA-1 calls them "a", "b", "c", &c).

</div>

As with Challenge 29, we'll do this inside our "evil" implementation of MD4.

In [3]:
original_msg = b'This is an authentic message!'
mac = MD4(original_msg).finish()

<div class="alert alert-block alert-info">

Modify your ~~SHA-1~~ **MD4** implementation so that callers can pass in new values for "a", "b", "c" &c (they normally start at magic numbers). With the registers "fixated", hash the additional data you want to forge.

</div>

In [4]:
import codecs
import struct

def leftrotate(i, n):
    return ((i << n) & 0xffffffff) | (i >> (32 - n))

def F(x, y, z):
    return (x & y) | (~x & z)

def G(x, y, z):
    return (x & y) | (x & z) | (y & z)

def H(x, y, z):
    return x ^ y ^ z

class evil_MD4(object):
    def __init__(self, data, state, prev_len):
        
        self.remainder = data
        self.prev_len = prev_len
        self.count = 0
        
        a = int.from_bytes(state[0:4], 'little')
        b = int.from_bytes(state[4:8], 'little')
        c = int.from_bytes(state[8:12], 'little')
        d = int.from_bytes(state[12:16], 'little')
        
        self.h = [a, b, c, d]

    def _add_chunk(self, chunk):
        self.count += 1
        X = list( struct.unpack("<16I", chunk) + (None,) * (80-16) )
        h = [x for x in self.h]
        # Round 1
        s = (3, 7, 11, 19)
        for r in range(16):
            i = (16-r)%4
            k = r
            h[i] = leftrotate( (h[i] + F(h[(i+1)%4], h[(i+2)%4], h[(i+3)%4]) + X[k]) % 2**32, s[r%4] )
        # Round 2
        s = (3, 5, 9, 13)
        for r in range(16):
            i = (16-r)%4
            k = 4*(r%4) + r//4
            h[i] = leftrotate( (h[i] + G(h[(i+1)%4], h[(i+2)%4], h[(i+3)%4]) + X[k] + 0x5a827999) % 2**32, s[r%4] )
        # Round 3
        s = (3, 9, 11, 15)
        k = (0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15) #wish I could function
        for r in range(16):
            i = (16-r)%4
            h[i] = leftrotate( (h[i] + H(h[(i+1)%4], h[(i+2)%4], h[(i+3)%4]) + X[k[r]] + 0x6ed9eba1) % 2**32, s[r%4] )

        for i, v in enumerate(h):
            self.h[i] = (v + self.h[i]) % 2**32

    def add(self, data):
        message = self.remainder + data
        r = len(message) % 64
        if r != 0:
            self.remainder = message[-r:]
        else:
            self.remainder = b''
        for chunk in range(0, len(message)-r, 64):
            self._add_chunk( message[chunk:chunk+64] )
        return self

    def finish(self):        
        l = self.prev_len + len(self.remainder) + 64 * self.count
        self.add( b'\x80' + b'\x00' * ((55 - l) % 64) + struct.pack("<Q", l * 8) )
        out = struct.pack("<4I", *self.h)
        ##self.__init__()
        return out

<div class="alert alert-block alert-info">

Using this attack, generate a secret-prefix MAC under a secret key (choose a random word from /usr/share/dict/words or something) of the string:

```"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"```

</div>

In [5]:
unknown_MD4_key = Random.get_random_bytes(32)

def make_MAC(message):
    
    my_md4 = MD4(unknown_MD4_key + bytes(message))
    MAC = my_md4.finish()
    
    return(MAC)

def check_MAC(message, MAC):
    
    return(make_MAC(message) == MAC)
    

In [6]:
original_msg = b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon'
original_mac = make_MAC(original_msg)

<div class="alert alert-block alert-info">

Forge a variant of this message that ends with ```";admin=true"```.

</div>

In [9]:
new_msg = b';admin=true'

FAKE_KEY = b'\x00' * 32
glue_padding = cp.compute_md4_padding(FAKE_KEY + original_msg)

last_len = (len(FAKE_KEY) + len(original_msg) + len(glue_padding)) 
if (last_len % 64) != 0:
    raise(ValueError('Invalid Length Calculated'))
    
forged_mac = evil_MD4(new_msg, original_mac, last_len).finish()
forged_msg = original_msg + glue_padding + new_msg

print(f"My forged message:\n")
print(forged_msg)
print()
print(f"MAC Check passed?  {check_MAC(forged_msg, forged_mac)}")

My forged message:

b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00h\x03\x00\x00\x00\x00\x00\x00;admin=true'

MAC Check passed?  True


Well that was easy.

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)